##### Silver Speech and Golden Silence: Spoiler Detection Project

### Sentence-wise Balanced Random Forest with Metadata (combined in 1 feature)

In [34]:
#Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier

#### Load Data

In [35]:
train = pd.read_json('data/train_reduced.json')
train.drop(['user_id', 'title'], inplace = True, axis = 1)
#Correct the review_length column
train['review_len'] = train['tokenized'].apply(lambda x: len(x))

In [36]:
val = pd.read_json('data/validation_reduced.json')
val.drop(['user_id', 'title'], inplace = True, axis = 1)
#Correct the review_length column
val['review_len'] = val['tokenized'].apply(lambda x: len(x))

In [37]:
#Import reviews as string (not in list)
X_tr = pd.read_csv('data/X_train.csv')
X_v = pd.read_csv('data/X_val.csv')
X_tr.rename(columns = {"0": 'review_whole'}, inplace = True)
X_v.rename(columns = {"0": 'review_whole'}, inplace = True)

In [38]:
#Combine data 
train = pd.concat([train, X_tr.review_whole], axis = 1)
val = pd.concat([val, X_v.review_whole], axis = 1)

In [39]:
train.to_json('data/train_only_needed_columns.json')
val.to_json('data/val_only_needed_columns.json')

In [40]:
#Since the data is very inbalanced, we only take the training reviews containing spoiler sentences. 
train = train[train['spoiler_dum']== 1]

In [42]:
train.reset_index(inplace = True, drop = True)

In [43]:
#Function to split the reviews into single sentences
def get_sentencewise_df(df):
    
    '''Get review sentences with according label, genre, review length from a dataframe df given in.
    The review sentences are written in a list 'lst' which is then transformed
    into a numpy array'''
    
    sen_lst = []
    genre_lst = []
    len_lst = []
    label_lst = []
    for i in range(len(df)):
        for sentence in df['tokenized'][i]:
            sen_lst.append(sentence)
            genre_lst.append(df['genre'][i])
            len_lst.append(df['review_len'][i])
            
    for labellist in df['sentence_labels']:
        for label in labellist:
            label_lst.append(label)
    df = pd.concat([pd.Series(sen_lst), pd.Series(label_lst), pd.Series(genre_lst), pd.Series(len_lst)], axis = 1)
    return df

In [44]:
#Train data sentence-wise
train = get_sentencewise_df(train)
train.rename(columns = {0: 'sentence', 1: 'spoiler_dum', 2: 'genre', 3: 'review_length'}, inplace = True)
train.head()

,sentence,spoiler_dum,genre,review_length
0,overall plot book get star maybe,0,"fantasy, paranormal",11
1,bad trilogy end,0,"fantasy, paranormal",11
2,feeling chase character aside basically feelin...,1,"fantasy, paranormal",11
3,unacceptable,1,"fantasy, paranormal",11
4,give book star go able read case later book se...,0,"fantasy, paranormal",11


In [45]:
#Validation data sentence-wise
val = get_sentencewise_df(val)
val.rename(columns = {0: 'sentence', 1: 'spoiler_dum', 2: 'genre', 3: 'review_length'}, inplace = True)
val.head()

,sentence,spoiler_dum,genre,review_length
0,look like promising new fantasy series,0,"fantasy, paranormal",3
1,love character definitely omg moment read,0,"fantasy, paranormal",3
2,negative wish term mythology explain little well,0,"fantasy, paranormal",3
3,definitely remind inuyasha read,0,"fantasy, paranormal",7
4,blend humour romance delightful,0,"fantasy, paranormal",7


In [46]:
#New column with genre and review text
train[['text_and_genre']] = train.genre + ' ' + train.sentence
val[['text_and_genre']] = val.genre + ' ' + val.sentence

In [47]:
X_train = train[['sentence', 'genre', 'review_length']]
X_train_rev = train.text_and_genre
y_train = train.spoiler_dum

In [48]:
X_val = val[['sentence', 'genre', 'review_length']]
X_val_rev = val.text_and_genre
y_val = val.spoiler_dum

#### Balanced Random Forest 

In [49]:
#Build a pipeline for feature extraction with TF IDF and SGD
#CountVectorizer
vectorizer = CountVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 100)

#Feature Selection: K-Best
feat = SelectKBest(score_func = chi2, k = 5000)

#Balanced Random Forest classifier
brf = BalancedRandomForestClassifier(n_estimators = 50, random_state = 42, verbose = 5, n_jobs = -1, class_weight = 'balanced', oob_score = True)

pipe = Pipeline([('count', vectorizer),('select_kbest', feat), ('BRF', brf)])

In [50]:
#Function for model training and prediction
def run_model(pipeline, X_train, y_train, X_test, y_test):
    #Fit the model
    model = pipeline.fit(X_train, y_train)
    
    #Predict labels of test data
    y_pred = model.predict(X_test)
    
    #Print classification report and confusion matrix
    return print(classification_report(y_test, y_pred)), print(confusion_matrix(y_test, y_pred, normalize = 'true'))
    return y_pred

In [51]:
#Run the model sentence-wise (validation data)
run_model(pipe, X_train_rev, y_train, X_val_rev, y_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 50
building tree 2 of 50
building tree 3 of 50
building tree 4 of 50
building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 14.5min


building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 69.6min finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/imblearn/ensemble/_forest.py:611: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    8.2s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   36.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 14.3min


building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 68.8min finished
/opt/anaconda3/envs/nf/lib/python3.6/site-packages/imblearn/ensemble/_forest.py:611: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   23.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  4.5min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:  4.6min finished
[Parallel(n_jobs=4)]: Using backe

              precision    recall  f1-score   support

           0       0.97      0.99      0.98   3396530
           1       0.29      0.12      0.17    113966

    accuracy                           0.96   3510496
   macro avg       0.63      0.56      0.58   3510496
weighted avg       0.95      0.96      0.95   3510496

[[0.98999391 0.01000609]
 [0.87781444 0.12218556]]


(None, None)